In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
import yfinance as yf
import os

### Historical Volatility

The histotical price of AMZN, Inc.

In [2]:
AMZN = yf.download("AMZN", start="2022-12-19", end="2023-12-19")

[*********************100%%**********************]  1 of 1 completed


In [3]:
S = AMZN['Adj Close'][-1]
print('The spot price is $', round(S,2), '.')

The spot price is $ 154.07 .


In [4]:
log_return = np.log(AMZN['Adj Close'] / AMZN['Adj Close'].shift(1))
vol_h = np.sqrt(252) * log_return.std()
print('The annualised volatility is', round(vol_h*100,2), '%')

The annualised volatility is 33.4 %


The Newton-Raphson Method to estimate impolied volatility

In [5]:
def newton_vol_call(S, K, T, C, r):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #C: Call value
    #r: risk free rate
    #sigma: volatility of underlying asset
   
    MAX_ITERATIONS = 1000
    tolerance = 0.000001
    
    sigma = 0.25
    
    for i in range(0, MAX_ITERATIONS):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        price = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
        vega = S * np.sqrt(T) * si.norm.pdf(d1, 0.0, 1.0)

        diff = C - price

        if (abs(diff) < tolerance):
            return sigma
        else: 
            sigma = sigma + diff/vega
        
        # print(i,sigma,diff)
        
    return sigma

Download the AMZN option data.

In [6]:
AMZN = yf.Ticker("AMZN")
opt = AMZN.option_chain('2024-01-19')
opt.calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AMZN240119C00045000,2023-12-18 19:49:38+00:00,45.0,109.67,108.45,109.15,0.000000,0.000000,1,316,1.998047,True,REGULAR,USD
1,AMZN240119C00050000,2023-12-15 16:32:00+00:00,50.0,99.41,103.50,104.20,0.000000,0.000000,5,1953,1.875001,True,REGULAR,USD
2,AMZN240119C00052000,2023-12-15 15:51:06+00:00,52.0,98.00,101.50,102.20,0.000000,0.000000,1,3296,1.813477,True,REGULAR,USD
3,AMZN240119C00053000,2023-12-14 16:19:03+00:00,53.0,95.50,100.50,101.20,0.000000,0.000000,9,1761,1.783204,True,REGULAR,USD
4,AMZN240119C00054000,2023-10-10 16:30:01+00:00,54.0,77.38,86.80,87.90,0.000000,0.000000,201,1401,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,AMZN240119C05000000,2022-06-03 17:49:58+00:00,5000.0,37.09,33.05,41.05,-1.979999,-5.067826,7,474,7.851319,False,REGULAR,USD
340,AMZN240119C05100000,2022-06-03 14:41:34+00:00,5100.0,31.87,30.30,38.05,0.930000,3.005819,2,108,7.704224,False,REGULAR,USD
341,AMZN240119C05200000,2022-06-03 13:51:00+00:00,5200.0,34.20,27.75,35.30,3.200001,10.322583,1,437,7.566163,False,REGULAR,USD
342,AMZN240119C05300000,2022-06-02 13:36:26+00:00,5300.0,29.00,25.65,32.95,0.000000,0.000000,1,698,7.450074,False,REGULAR,USD


In [7]:
impvol = newton_vol_call(S, 165, 4/52, float(opt.calls.lastPrice[opt.calls.strike == 165.00]), 0.013675)
print('The implied volatility is', round(impvol*100,2) , '% for the one-month call with strike $ 165.00' ) 

The implied volatility is 25.47 % for the one-month call with strike $ 165.00


C:\Users\ibiro\AppData\Local\Temp\ipykernel_19816\2574922674.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  impvol = newton_vol_call(S, 165, 4/52, float(opt.calls.lastPrice[opt.calls.strike == 165.00]), 0.013675)
